In [26]:
NEW_ADMISSION_IDS = []
files = ['list_4000.txt']
for file in files:
    f = open('/home/kanit/mimic3/'+file, 'r')
    adms = f.readlines();
    for adm in adms:
        NEW_ADMISSION_IDS.append(int(adm))
ADMISSION_IDS = NEW_ADMISSION_IDS
print len(ADMISSION_IDS)

# just set 'ADMISSION_IDS' for code to work

4000


In [27]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
try:
    # conn = psycopg2.connect("dbname='mimic' user='kanit' host='melady1.usc.edu' password='abcd6712'")
    conn = psycopg2.connect("dbname='mimic' user='kanit' host='localhost' password='abcd6712'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


In [28]:
cate = ['admission_type','admission_location','insurance','language','religion','marital_status','ethnicity']
mapping = {}
for c in cate:
    cur = conn.cursor()
    cur.execute('select distinct '+c+' from mimiciii.admissions')
    types = cur.fetchall()
    
    catemapping = {}
    for i in range(len(types)):
        catemapping[types[i][0]] = i 
    mapping[c] = catemapping;

In [29]:
DB = []
ct =0 ;
for aid in ADMISSION_IDS:
    print str(ct) + " : " + str(aid)
    ct+=1
    
    cur = conn.cursor()
    cur.execute('SELECT hadm_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity FROM mimiciii.ADMISSIONS WHERE hadm_id='+str(aid))
    admission = cur.fetchone()
    
    assert admission!=None
    
    subject_id = admission[1]
    admittime = admission[2]
    dischtime = admission[3]
    deathtime = admission[4]
    
    cur = conn.cursor()
    cur.execute('SELECT dob FROM mimiciii.PATIENTS WHERE subject_id='+str(subject_id))
    patient = cur.fetchone()
    
    assert patient!=None
    birthdate = patient[0]
    
    mortal = 0
    labelGuarantee = 0
    if(deathtime != None) : 
        mortal = 1
        if(deathtime != dischtime):
            labelGuarantee=1

    data = [aid,subject_id, (admittime - birthdate).total_seconds()/(3600*24), (dischtime-admittime).total_seconds()//60, mortal, labelGuarantee]
    for i in range(5,12):
        data.append(mapping[cate[i-5]][admission[i]])
    data.extend([birthdate,admittime,dischtime])
    DB.append(data)
    

0 : 117057
1 : 174526
2 : 148079
3 : 165651
4 : 196480
5 : 131525
6 : 107820
7 : 178984
8 : 136802
9 : 158978
10 : 118505
11 : 107871
12 : 115485
13 : 182520
14 : 130226
15 : 138692
16 : 165355
17 : 181515
18 : 176448
19 : 137691
20 : 119103
21 : 164940
22 : 100357
23 : 128490
24 : 164091
25 : 159603
26 : 132798
27 : 198661
28 : 112602
29 : 173623
30 : 115825
31 : 143639
32 : 182958
33 : 136210
34 : 107776
35 : 155976
36 : 127630
37 : 113413
38 : 170066
39 : 148883
40 : 118447
41 : 110313
42 : 120251
43 : 113693
44 : 132536
45 : 194675
46 : 153420
47 : 124971
48 : 143877
49 : 167199
50 : 154956
51 : 161287
52 : 186096
53 : 138276
54 : 174733
55 : 115795
56 : 105804
57 : 134230
58 : 181341
59 : 137937
60 : 196870
61 : 160894
62 : 134016
63 : 192384
64 : 189442
65 : 184704
66 : 125899
67 : 128681
68 : 177708
69 : 170129
70 : 149479
71 : 196309
72 : 147813
73 : 199125
74 : 149199
75 : 150561
76 : 176481
77 : 139036
78 : 145527
79 : 113458
80 : 175502
81 : 156694
82 : 164964
83 : 179381
84

In [30]:
# save it
import numpy as np;
np.save('AGE_LOS_MORTALITY_4000',DB)

In [31]:
DB[0]

[117057,
 21514,
 22637.03888888889,
 8200.0,
 0,
 0,
 3,
 0,
 2,
 4,
 0,
 5,
 28,
 datetime.datetime(2047, 9, 9, 0, 0),
 datetime.datetime(2109, 9, 1, 0, 56),
 datetime.datetime(2109, 9, 6, 17, 36)]